In [37]:
libraries = c("dplyr","magrittr","tidyr","ggplot2","rstan","readxl")
for(x in libraries) { library(x,character.only=TRUE,warn.conflicts=FALSE,quietly=TRUE) }

require(zoo)
require(lubridate)

base_sz = 12 # base_size parameter
theme_set(theme_bw())

'%&%' = function(x,y) paste0(x,y)

options(mc.cores = parallel::detectCores())
rstan_options(auto_write = TRUE)

packageVersion("rstan")
packageVersion("StanHeaders")
rstan::stan_version()

[1] ‘2.19.2’

[1] ‘2.21.0.1’

[1] "2.19.1"

# <font color="purple">Preprocessing of the data</font>

In [38]:
# datafilename = "../../data/wuhan_data_master_20200125_6pm.xlsx"
datafilename = "../../../Hokkaido_Wuhan Data 2020/wuhan_data_master.xlsx"

In [39]:
read_excel(datafilename, sheet="export") -> Df
names(Df)

[1] "ID"                            "DeathID"                      
 [3] "Age"                           "AgeDecade"                    
 [5] "Sex"                           "InWuhan2WeeksBeforeOnset"     
 [7] "ExposureL"                     "ExposureR"                    
 [9] "ExposureType"                  "Onset"                        
[11] "DateCaseFirstSoughtHealthcare" "DateHospIsolated"             
[13] "HospOrIsol"                    "DateOfDeath"                  
[15] "DateDischarged"                "DateReportedConfirmed"        
[17] "ResidenceCity"                 "ResidenceProvince"            
[19] "OfficialResidenceCity"         "OfficialResidenceProvince"    
[21] "ResidenceCountry"              "Nationality"                  
[23] "DiagnosisCity"                 "DiagnosisProvince"            
[25] "DiagnosisCountry"              "LocalCaseNum"                 
[27] "Pneumonia"                     "WuhanFreshMarket"             
[29] "WuhanSCSCVisit"                "DateSpecColl"                 
[31] "LabPosDate"                    "CaseClass"                    
[33] "Evacuaee"                      "CruiseShip"                   
[35] "URL"                           "Note"

## <font color="purple">Parameters</font>

In [40]:
### day zero
t0 = as.Date("2019-12-09")

#### setting the estimated window time
detection_window = 12.5

#### setting the population size of Wuhan
popWuhan = 11081000

#### setting the volume of inbound passengers from China per year
total_travellers = 55568293

#### fraction of Wuhan travellers in the total flow of travellers from China
fraction_travellers_Wuhan = 0.020763

In [41]:
#### probability of travel outside of Mainland China
prob_travel = (total_travellers*fraction_travellers_Wuhan*detection_window)/(365*popWuhan)

prob_HN = 365/(total_travellers*fraction_travellers_Wuhan*detection_window)

**IMPORTANT**

In [42]:
#### Cut-off time for our analysis
CUTOFF_DATE = as.Date("2020-01-24")

## <font color="orange">Creating skeleton for the resulting data frame</font>

In [43]:
df = data.frame(date = as.Date(t0:CUTOFF_DATE), prob_travel = prob_travel, prob_HN = prob_HN, popWuhan = popWuhan) %>% mutate(time = 1:n())
head(df)

,date,prob_travel,prob_HN,popWuhan,time
,<date>,<dbl>,<dbl>,<dbl>,<int>
1,2019-12-09,0.003565787,2.530846e-05,11081000,1
2,2019-12-10,0.003565787,2.530846e-05,11081000,2
3,2019-12-11,0.003565787,2.530846e-05,11081000,3
4,2019-12-12,0.003565787,2.530846e-05,11081000,4
5,2019-12-13,0.003565787,2.530846e-05,11081000,5
6,2019-12-14,0.003565787,2.530846e-05,11081000,6


## <font color="orange">Counts of exported cases by date of reporting</font>

In [44]:
df_exports = read_excel(datafilename, sheet="export") %>% 
    filter(DiagnosisCountry!='China') %>% 
    select(DiagnosisCountry,Onset,DateReportedConfirmed) %>%
    rename(`Confirmed`=`DateReportedConfirmed`, `Country`=`DiagnosisCountry`) %>%
    arrange(Confirmed,Onset) %>%
    filter(as.Date(Confirmed) <= CUTOFF_DATE)
#     mutate(time = as.numeric(as.Date(DateReportedConfirmed)-t0)) 
df_exports 

Country,Onset,Confirmed
<chr>,<dttm>,<dttm>
Thailand,2020-01-05,2020-01-13
Japan,2020-01-03,2020-01-16
Thailand,NA,2020-01-17
South Korea,2020-01-18,2020-01-20
USA,2020-01-15,2020-01-21
Taiwan,NA,2020-01-21
Thailand,NA,2020-01-22
Thailand,NA,2020-01-22
Vietnam,2020-01-17,2020-01-23


In [45]:
df_exports %>% group_by(Country) %>% summarize(count = n())

Country,count
<chr>,<int>
Japan,2
Nepal,1
Singapore,3
South Korea,2
Taiwan,3
Thailand,5
USA,2
Vietnam,2


In [46]:
df_exports %>% nrow

[1] 20

In [47]:
mean_reporting_delay = df_exports %>% na.omit() %>% mutate(diff = as.Date(Confirmed)-as.Date(Onset)) %>% .$diff %>% mean
mean_reporting_delay

Time difference of 5.333333 days

In [48]:
df_exports %<>% mutate(ImputedWithMeanDelay = if_else(is.na(Onset), TRUE, FALSE), 
                     Onset = if_else(is.na(Onset), lubridate::floor_date(Confirmed - mean_reporting_delay, unit = 'days'), Onset))
df_exports %>% arrange(Confirmed,Onset) %>% mutate(id = 1:n()) %>% select(id, Country, everything()) 

id,Country,Onset,Confirmed,ImputedWithMeanDelay
<int>,<chr>,<dttm>,<dttm>,<lgl>
1,Thailand,2020-01-05,2020-01-13,FALSE
2,Japan,2020-01-03,2020-01-16,FALSE
3,Thailand,2020-01-11,2020-01-17,TRUE
4,South Korea,2020-01-18,2020-01-20,FALSE
5,USA,2020-01-15,2020-01-21,FALSE
6,Taiwan,2020-01-15,2020-01-21,TRUE
7,Thailand,2020-01-16,2020-01-22,TRUE
8,Thailand,2020-01-16,2020-01-22,TRUE
9,Vietnam,2020-01-17,2020-01-23,FALSE


In [49]:
data_drname = "../../data"
flname = 'data_exports.csv'
write.table(df_exports, paste0(data_drname,flname), row.names=FALSE, sep=",", quote = FALSE)

**Adding exportation cases to the resulting data frame**

In [50]:
df_exports %>% 
    select(Confirmed) %>% 
    group_by(Confirmed) %>% 
    summarize(exports=n()) %>%
    rename(`date`=`Confirmed`) %>%
    mutate(date = as.Date(date)) %>%
    mutate(exports=cumsum(exports)) %>%
    right_join(df, by='date') -> df

df$exports[1] = 0
df$exports = zoo::na.locf(df$exports, fromLast=FALSE)

tail(df,10)

date,exports,prob_travel,prob_HN,popWuhan,time
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
2020-01-15,1,0.003565787,2.530846e-05,11081000,38
2020-01-16,2,0.003565787,2.530846e-05,11081000,39
2020-01-17,3,0.003565787,2.530846e-05,11081000,40
2020-01-18,3,0.003565787,2.530846e-05,11081000,41
2020-01-19,3,0.003565787,2.530846e-05,11081000,42
2020-01-20,4,0.003565787,2.530846e-05,11081000,43
2020-01-21,6,0.003565787,2.530846e-05,11081000,44
2020-01-22,8,0.003565787,2.530846e-05,11081000,45
2020-01-23,11,0.003565787,2.530846e-05,11081000,46


## <font color="orange">Death cases by date of reporting</font>

In [51]:
read_excel(datafilename, sheet="deaths") %>% 
    select(DateOfDeath) %>% 
    group_by(DateOfDeath) %>% 
    summarize(deaths=n()) %>%
    na.omit() -> df_deaths

sum(df_deaths$deaths)

[1] 46

In [52]:
df_deaths %>%
    rename(date=DateOfDeath) %>%
    mutate(date=as.Date(date)) %>%
    arrange(date) %>% 
    mutate(deaths = cumsum(deaths)) %>%
    right_join(df, by="date") -> df

df$deaths[1] = 0
df$deaths = zoo::na.locf(df$deaths, fromLast=FALSE)

tail(df,10)

date,deaths,exports,prob_travel,prob_HN,popWuhan,time
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
2020-01-15,2,1,0.003565787,2.530846e-05,11081000,38
2020-01-16,2,2,0.003565787,2.530846e-05,11081000,39
2020-01-17,2,3,0.003565787,2.530846e-05,11081000,40
2020-01-18,3,3,0.003565787,2.530846e-05,11081000,41
2020-01-19,4,3,0.003565787,2.530846e-05,11081000,42
2020-01-20,7,4,0.003565787,2.530846e-05,11081000,43
2020-01-21,18,6,0.003565787,2.530846e-05,11081000,44
2020-01-22,24,8,0.003565787,2.530846e-05,11081000,45
2020-01-23,34,11,0.003565787,2.530846e-05,11081000,46


## <font color="orange">Official case counts</font>

In [53]:
read_excel(datafilename, sheet="cases_CHN") %>% 
    select(Date,Cases) %>%
    rename(`date`=`Date`, `reported`=`Cases`) %>%
    mutate(date=as.Date(date)) %>%
    arrange(date) %>%
    right_join(df) -> df

df$reported[1] = 0
df$reported = zoo::na.locf(df$reported, fromLast=FALSE)

tail(df, 10)

Joining, by = "date"



date,reported,deaths,exports,prob_travel,prob_HN,popWuhan,time
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
2020-01-15,41,2,1,0.003565787,2.530846e-05,11081000,38
2020-01-16,45,2,2,0.003565787,2.530846e-05,11081000,39
2020-01-17,62,2,3,0.003565787,2.530846e-05,11081000,40
2020-01-18,121,3,3,0.003565787,2.530846e-05,11081000,41
2020-01-19,198,4,3,0.003565787,2.530846e-05,11081000,42
2020-01-20,291,7,4,0.003565787,2.530846e-05,11081000,43
2020-01-21,440,18,6,0.003565787,2.530846e-05,11081000,44
2020-01-22,571,24,8,0.003565787,2.530846e-05,11081000,45
2020-01-23,830,34,11,0.003565787,2.530846e-05,11081000,46


In [54]:
head(df, 10)

date,reported,deaths,exports,prob_travel,prob_HN,popWuhan,time
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
2019-12-09,0,0,0,0.003565787,2.530846e-05,11081000,1
2019-12-10,0,0,0,0.003565787,2.530846e-05,11081000,2
2019-12-11,0,0,0,0.003565787,2.530846e-05,11081000,3
2019-12-12,0,0,0,0.003565787,2.530846e-05,11081000,4
2019-12-13,0,0,0,0.003565787,2.530846e-05,11081000,5
2019-12-14,0,0,0,0.003565787,2.530846e-05,11081000,6
2019-12-15,0,0,0,0.003565787,2.530846e-05,11081000,7
2019-12-16,0,0,0,0.003565787,2.530846e-05,11081000,8
2019-12-17,0,0,0,0.003565787,2.530846e-05,11081000,9


### <font color="orange">Saving the dataframe</font>

In [55]:
## re-arranging the column order
df %<>% select(date, time, everything())

In [56]:
flname = '../../data/data.csv'
write.table(df, flname, row.names=FALSE, sep=",", quote = FALSE)

## <font color="orange">Data: onset to death </font>

In [57]:
read_excel(datafilename, sheet="deaths") %>% 
    select(Onset, DateOfDeath) %>%
    mutate(dist = as.numeric(as.Date(DateOfDeath) - as.Date(Onset)),
           distUpper = as.numeric(CUTOFF_DATE - as.Date(Onset))) %>%
    na.omit() %>% 
    filter(as.Date(DateOfDeath) <= CUTOFF_DATE) %>%
    select(Onset,distUpper,dist) -> df_onset2death

df_onset2death %>% write.table('../../data/data_onset2death.csv', row.names=FALSE, sep=",", quote = FALSE)
df_onset2death

Onset,distUpper,dist
<dttm>,<dbl>,<dbl>
2019-12-20,35,20
2019-12-31,24,15
2020-01-13,11,6
2020-01-10,14,10
2020-01-06,18,14
2019-12-10,45,41
2020-01-09,15,12
2019-12-22,33,30
2020-01-15,9,7


## <font color="orange">Data: onset to reporting for overseas travellers</font>

In [58]:
read_excel(datafilename, sheet="export") %>%
    filter(DiagnosisCountry!='China') %>% nrow

[1] 249

In [59]:
read_excel(datafilename, sheet="export") %>%
    filter(DiagnosisCountry!='China') %>%
    select(Onset, DateReportedConfirmed) %>%
    mutate(dist = as.numeric(as.Date(DateReportedConfirmed) - as.Date(Onset)),
           distUpper = as.numeric(CUTOFF_DATE - as.Date(Onset))) %>%
    filter(as.Date(Onset) <= CUTOFF_DATE) -> df_onset2report

df_onset2report

Onset,DateReportedConfirmed,dist,distUpper
<dttm>,<dttm>,<dbl>,<dbl>
2020-01-03,2020-01-16,13,21
2020-01-05,2020-01-13,8,19
2020-01-18,2020-01-20,2,6
2020-01-15,2020-01-21,6,9
2020-01-17,2020-01-23,6,7
2020-01-20,2020-01-23,3,4
2020-01-14,2020-01-24,10,10
2020-01-21,2020-01-23,2,3
2020-01-10,2020-01-24,14,14


In [60]:
df_onset2report %<>%
    na.omit() %>% 
    select(Onset,distUpper,dist) -> df_onset2report

df_onset2report %>% write.table('../../data/data_onset2report.csv', row.names=FALSE, sep=",", quote = FALSE)
df_onset2report %>% head

Onset,distUpper,dist
<dttm>,<dbl>,<dbl>
2020-01-03,21,13
2020-01-05,19,8
2020-01-18,6,2
2020-01-15,9,6
2020-01-17,7,6
2020-01-20,4,3
